In [10]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

import  yfinance as yf

In [11]:
# Get historical OHLCV data for target ticker
etf_df = yf.download(
    "STW.AX, VAS.AX", 
    period="max"
)

[*********************100%***********************]  2 of 2 completed


In [12]:
# Review data to see the date ranges
display(etf_df.head())
display(etf_df.tail())
display(etf_df.shape)
display(etf_df.index)


Adj Close             Close              High               Low  \
               STW.AX VAS.AX     STW.AX VAS.AX     STW.AX VAS.AX     STW.AX   
Date                                                                          
2008-01-02  61.169998    NaN  61.169998    NaN  61.169998    NaN  61.169998   
2008-01-03  60.590000    NaN  60.590000    NaN  60.590000    NaN  60.590000   
2008-01-04  60.740002    NaN  60.740002    NaN  60.740002    NaN  60.740002   
2008-01-07  59.349998    NaN  59.349998    NaN  59.349998    NaN  59.349998   
2008-01-08  59.020000    NaN  59.020000    NaN  59.020000    NaN  59.020000   

                        Open        Volume         
           VAS.AX     STW.AX VAS.AX STW.AX VAS.AX  
Date                                               
2008-01-02    NaN  61.169998    NaN      0    NaN  
2008-01-03    NaN  60.590000    NaN      0    NaN  
2008-01-04    NaN  60.740002    NaN      0    NaN  
2008-01-07    NaN  59.349998    NaN      0    NaN  
2008-01-08    NaN  59.020000    NaN      0    NaN

Adj Close                 Close                  High             \
               STW.AX     VAS.AX     STW.AX     VAS.AX     STW.AX     VAS.AX   
Date                                                                           
2022-12-16  65.010002  88.980003  65.010002  88.980003  65.339996  89.440002   
2022-12-19  64.849998  88.800003  64.849998  88.800003  65.040001  89.050003   
2022-12-20  63.900002  87.440002  63.900002  87.440002  64.809998  88.709999   
2022-12-21  64.669998  88.500000  64.669998  88.500000  64.980003  88.839996   
2022-12-22  65.040001  88.949997  65.040001  88.949997  65.199997  89.180000   

                  Low                  Open             Volume             
               STW.AX     VAS.AX     STW.AX     VAS.AX  STW.AX     VAS.AX  
Date                                                                       
2022-12-16  64.730003  88.570000  64.879997  88.800003  211859   155356.0  
2022-12-19  64.709999  88.570000  64.730003  88.589996   65356   239837.0  
2022-12-20  63.799999  87.300003  64.809998  88.570000  125904   357111.0  
2022-12-21  64.430000  88.129997  64.430000  88.139999   69366   154546.0  
2022-12-22  65.019997  88.879997  65.110001  89.000000   52931  1206436.0

(3790, 12)

DatetimeIndex(['2008-01-02', '2008-01-03', '2008-01-04', '2008-01-07',
               '2008-01-08', '2008-01-09', '2008-01-10', '2008-01-11',
               '2008-01-14', '2008-01-15',
               ...
               '2022-12-09', '2022-12-12', '2022-12-13', '2022-12-14',
               '2022-12-15', '2022-12-16', '2022-12-19', '2022-12-20',
               '2022-12-21', '2022-12-22'],
              dtype='datetime64[ns]', name='Date', length=3790, freq=None)

In [9]:
# Examine the column type and check for nulls
etf_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3790 entries, 2008-01-02 to 2022-12-22
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3790 non-null   float64
 1   High       3790 non-null   float64
 2   Low        3790 non-null   float64
 3   Close      3790 non-null   float64
 4   Adj Close  3790 non-null   float64
 5   Volume     3790 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 207.3 KB


In [13]:
# Retain the key columns for each etf
etf_df = etf_df.drop(columns = ['Adj Close', 'High', 'Low', 'Open', 'Volume'])
etf_df.columns = etf_df.columns.droplevel()     # Changes the multilevel indexing on columns to single level
etf_df.tail()

,STW.AX,VAS.AX
Date,,
2022-12-16,65.010002,88.980003
2022-12-19,64.849998,88.800003
2022-12-20,63.900002,87.440002
2022-12-21,64.669998,88.500000
2022-12-22,65.040001,88.949997
